### Read weights from hdf5 file and convert them into bit vectors (for FPGA implementation)

In [23]:
filename = 'E:\\deeplearning\\Hepatocarcinomes\\models\\5x\\vgg16_dense_bn\\weights.15-0.8841.hdf5'

In [24]:
import h5py

f = h5py.File(filename, 'r')

#List all groups
print(type(f[list(f.keys())[0]])) #Groups are like Python dictionaries. traverse down till reaching a dataset
print("Keys: %s" % f.keys())

<class 'h5py._hl.group.Group'>
Keys: <KeysViewHDF5 ['activation_1', 'activation_2', 'block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool', 'dense1_bn', 'dense2_bn', 'dense_1', 'dense_2', 'dense_3', 'dropout_1', 'dropout_2', 'flatten_1', 'input_1']>


In [25]:
#extract all dataset names 
def traverse_datasets(hdf_file):

    def h5py_dataset_iterator(g, prefix=''):
        for key in g.keys():
            item = g[key]
            path = f'{prefix}/{key}'
            if isinstance(item, h5py.Dataset): # test for dataset
                yield (path, item)
            elif isinstance(item, h5py.Group): # test for group (go down)
                yield from h5py_dataset_iterator(item, path)

    with h5py.File(hdf_file, 'r') as f:
        for path, _ in h5py_dataset_iterator(f):
            yield path

In [26]:
dset=traverse_datasets(filename)

In [30]:
print(next(dset))

/block1_conv2/block1_conv2/kernel:0


In [32]:
f[next(dset)]

<HDF5 dataset "kernel:0": shape (3, 3, 64, 128), type "<f4">

In [ ]:
#seave weights into (8+24) bit vectors
import math
with open("E:\\test\\dense2.txt", "w") as text_file:
    for j in range(f['dense_2/dense_2/kernel:0'].shape[1]):
        for i in range(f['dense_2/dense_2/kernel:0'].shape[0]):
            a,b=math.modf(f['dense_2/dense_2/kernel:0'][i, j])
            if b < 0:
                inte = str(bin(int(b))[3:])
                if len(inte) <= 7:
                    inte = '1'+inte.zfill(7)
                else:
                    inte = '1'+inte[:7]
            else:
                inte = str(bin(int(b))[2:])
                if len(inte) <= 8:
                    inte = inte.zfill(8)
                else:
                    inte = inte[:8]
            frac = str(bin(int(math.modf(abs(a)*10e8)[1]))[2:]).zfill(24)
            if len(frac) <= 24:
                frac = frac.zfill(24)
            else:
                frac = frac[:24]
            text_file.write(inte+frac+',')